In [1]:
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import round as rnd
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.pipeline import PipelineModel

In [2]:
#------------------------------------------Import SparkConf class into program------------------------------------------
from pyspark import SparkConf
from pyspark.sql.functions import isnan, when, count, col
master = "local[*]"
app_name = "Assignment Module"
spark_conf = SparkConf().setMaster(master).setAppName(app_name).set("spark.sql.files.maxPartitionBytes", 30000000)
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')
import matplotlib.pyplot as plt

21/10/14 08:25:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/14 08:25:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/14 08:25:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
#load from parquet
flightsDf_model = spark.read.parquet("flightsDf_model.parquet")
flightsDf = spark.read.parquet("flightsDf.parquet")


In [7]:
train_df, test_df = flightsDf_model.randomSplit([0.8, 0.2], seed=7)

In [4]:
persistedModelArrDT = PipelineModel.load('/home/jovyan/work/MarkingConsistency/Model/Arr')

In [5]:
persistedModelDeptDT = PipelineModel.load('/home/jovyan/work/MarkingConsistency/Model/Dept')

In [8]:
predicted_data_dt_arr_del = persistedModelArrDT.transform(test_df)
predicted_data_dt_dep_del = persistedModelDeptDT.transform(test_df)

In [9]:
# show confusion matrix
print("Confusion Matrix: Decision Tree: Arrival Delay")
predicted_data_dt_arr_del.groupBy('binaryArrDelay', 'prediction').count().withColumnRenamed("binaryArrDelay","label").show()


# show confusion matrix
print("Confusion Matrix: Decision Tree: Departure Delay")
predicted_data_dt_dep_del.groupBy('binaryDeptDelay', 'prediction').count().withColumnRenamed("binaryDeptDelay","label").show()


Confusion Matrix: Decision Tree: Arrival Delay


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       1.0|21298|
+-----+----------+-----+

Confusion Matrix: Decision Tree: Departure Delay
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       1.0|19357|
|    0|       1.0| 1941|
+-----+----------+-----+



In [10]:
def Model_Statistics(data,algo,name):
    
    print("Model Statistics for ",name," using ",algo,":")
    
    arr_stat_algo=[]
    
    global arr_stat
    
    data.cache()
    #------------------------------Calculate the elements of the confusion matrix----------------------------
    TN = data.filter('prediction = 0 AND label = 0').count() #---True Negative
    TP = data.filter('prediction = 1 AND label = 1').count() #---True Positive
    FP = data.filter('prediction = 1 AND label = 0').count() #---False Positive
    FN = data.filter('prediction = 0 AND label = 1').count() #---False Negative

    evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
    auc = evaluator.evaluate(data)

    accuracy = (TP+TN)/(TP+TN+FP+FN)
    recall = TP/(TP+FN)
    precision = TP/(TP+FP)
    f1 = 2/(1/recall+1/precision)
    
    arr_stat_algo.append(TP)
    arr_stat_algo.append(FN)
    arr_stat_algo.append(FP)
    arr_stat_algo.append(TN)
    arr_stat.append(arr_stat_algo)

    print("Accuarcy:",round(accuracy,2))
    print("Recall:",round(recall,2))
    print("Precision:",round(precision,2))
    print("F1:",round(f1,2))
    print("Area Under the Curve (AUC):",round(auc,2))
    print("-------------------------------------------------------------")
    print()

In [11]:
#-------------------------------------Renaming Columns--------------------------------

predicted_data_dt_arr_del=predicted_data_dt_arr_del.withColumnRenamed("binaryArrDelay","label")

predicted_data_dt_dep_del=predicted_data_dt_dep_del.withColumnRenamed("binaryDeptDelay","label")


 #------------------------------Defining Lists to Iterate----------------------------

list_data=[predicted_data_dt_arr_del, predicted_data_dt_dep_del]

list_name=["Arrival Delay","Departure Delay"]

list_algo=["Decision Tree","Decision Tree"]

arr_stat=[]

i=0

 #------------------------------Calculate statistics----------------------------

for i in range(0,2):
    
    Model_Statistics(list_data[i],list_algo[i],list_name[i])#---Calling Model_Statistics()

Model Statistics for  Arrival Delay  using  Decision Tree :
Accuarcy: 1.0
Recall: 1.0
Precision: 1.0
F1: 1.0
Area Under the Curve (AUC): 1.0
-------------------------------------------------------------

Model Statistics for  Departure Delay  using  Decision Tree :
Accuarcy: 0.91
Recall: 1.0
Precision: 0.91
F1: 0.95
Area Under the Curve (AUC): 0.5
-------------------------------------------------------------

